# ca1 : الگوریتم ژنتیک
#### mahdi vajhi
#### 810101558
<p dir=rtl style="text-align:right">
هدف از این پروژه حل مسئله 
Problem Knapsack
با استفاده از الگوریتم ژنتیک است. ما در این مسئله باید حالت بهینه از غذا ها را که شرایط ورودی را داشته باشد پیدا کنیم.
برای این کار ما ابتدا باید ساختار کروموزوم های خود را تعریف کنیم که دارای قابلیت تلفیق با بقیه و جهش را داشته باشد. سپس باید تعدادی کروموزوم اولیه تولید کنیم و با پیاده سازی یک تابع فیتنس برای آنها میزان خوب بودن آنها را بسنجیم این تابع باید به شرایط مسئله حساس باشد و همچنین میزان خوب بودن ژن ها را نشان دهد. سپس باید تابعی برای تولید نسل بعدی نوشت که ابتدا کروموزوم ها را جفت کند سپس آنها را تلفیق کند و در نهایت آنها را جهش دهد. در آخر هم باید نتایج را بررسی کرد و جواب به دست آمده را بررسی کنیم.
</p>

## initial block

In [240]:
import random
import pandas as pd
import numpy as np
import copy


GEN_ACTIVE_RATE = 0.5
WIGHT_MUTATION_RATE = 0.05
MIN_WIGHT = 0.5
IS_WIGHT_MUTATION_IF_DEACTIVATE = True
FIRST_CHROMOSOMES_COUNT = 10

## تعریف کروموزوم و ژن ها
<p dir=rtl>
در این مسئله با توجه به این که طول کروموزوم باید ثابت باشد و همچنین تعداد غذا ها ثابت است می توانیم کروموزوم را فهرست تمام غذا ها بگذاریم که با یک متغییر مشخص شود که آیا آن غذا را می بریم یا خیر و اگر می بریم چه مقدار از آن را می بریم. پس در این مسئله ژن ها همان غذا ها هستند که می تواند فعال یا غیر فعال باشند.
</p>

### ژن ها
<p dir=rtl>
کلاس ژن در واقع نماینده یک غذا است که در آن نام، حداکثر مقدار موجود، فعال بودن آن، چگالی ارزش آن و وزن الان آن است.
</p>
<p dir=rtl>
در تابع جهش ژن با احتمالی که تعیین شده می تواند فعال یا غیر فعال شود
همچنین وزن آن با یک توزیع نرمال حول مقدار قبلی خود تغییر می کند. توزیع به گونه ای تنظیم شده که انحراف معیار به اندازه یک ثابت بین ۰ تا ۱ در مقدار وزن باشد که تغییر معقول باشد. همچنین شرطی گذاشته شده که آیا درموقع غیر فعال بودن ژن وزن آن جهش کند یا خیر
</p>


In [241]:
class Gen:
    def __init__(self, name:str, max_weight:float, value_per_weight:float
                 , weight:float=0, is_active:bool=False):
        self.name = name
        self.max_weight = max_weight
        self.vpw = value_per_weight
        self.is_active = is_active
        self.weight = weight

    def set_random_gen(self):
        self.set_random_gen_status()
        self.set_random_wight()
    def set_random_gen_status(self):
        self.is_active = random.choice([True, False])
    def set_random_wight(self):
        self.weight = round(random.uniform(MIN_WIGHT, self.max_weight),1)

    def mutation(self):
        if IS_WIGHT_MUTATION_IF_DEACTIVATE or self.is_active:
            self.weight_mutation()
        self.active_mutation()
    def active_mutation(self):
        self.is_active = self.is_active if random.random() < GEN_ACTIVE_RATE\
                    else not self.is_active
    def weight_mutation(self):
        std = self.weight * WIGHT_MUTATION_RATE
        self.weight = np.random.normal(self.weight, std, 1)[0]
        # fix number in range and round
        self.weight = round((max(MIN_WIGHT, min(self.max_weight, self.weight))),1)
        
    # Getters
    def get_weight(self):
        return self.max_weight if self.is_active else 0
    def get_value(self):
        return self.max_weight * self.vpw if self.is_active else 0
    def get_name(self):
        return self.name
    def get_is_active(self):
        return self.is_active
    def __str__(self):
        return "------------\n" + f"name: {self.name}\nstatus: {self.is_active}\n" +\
        f"weight: {self.weight}\nmax weight: {self.max_weight}\nvpw: {self.vpw:.2f}"
    def __repr__(self):
        return self.__str__()

### کروموزوم ها
<p dir=rtl>
کلاس کروموزوم صرفا اجماع ژن ها است و توابع آن صرفا روی ژن ها حلقه می زند.
</p>

In [242]:
class Chromosome:
    def __init__(self, gens:list[Gen]):
        self.gens = gens

    def mutation(self):
        for i in self.gens:
            i.mutation()

    # Getters
    def get_weight(self):
        return sum([i.get_weight() for i in self.gens])
    def get_value(self):
        return sum([i.get_value() for i in self.gens])
    def get_vpw(self):
        return self.get_value() / self.get_weight()
    def get_name(self):
        return [i.get_name() for i in self.gens]
    def get_active_count(self):
        return sum([i.get_is_active() for i in self.gens])
    
    def __str__(self):
        return "__________\n"+\
                f"active_count: {self.get_active_count()}\n"+\
                f"weight: {self.get_weight()}\n"+\
                f"value: {self.get_value()}\n"+\
                f"vpw: {self.get_vpw():.2f}"
    def __repr__(self):
        return self.__str__()

## تولید جمعیت اولیه
<p dir=rtl>
در این قسمت باید جامعه اولیه را تولید کنیم که شامل مراجل زیر است:
</p>
<ol dir=rtl>
    <li> خواندن داده ها </li>
    <li> تولید ژن خام از غذا ها </li>
    <li> تولید کرموزوم ها با ژن های خام و مقدار دهی شانسی وزن آنها و وضعیت شانسی فعال بودن ژن ها </li>
</ol>

### read data
<p dir=rtl style="text-align:right">
در این قسمت فایل
snacks.csv
را لود می کنیم که داده های غذا های مربوط به پروژه در آن قرار دارد
</p>

In [243]:
snacks = pd.read_csv('./snacks.csv')
snacks

,Snack,Available Weight,Value
0,MazMaz,10,10
1,Doogh-e-Abali,15,10
2,Nani,5,5
3,Jooj,7,15
4,Hot-Dog,20,15
5,Chips,8,6
6,Nooshaba,12,8
7,Shokolat,6,7
8,Chocoroll,9,12
9,Cookies,11,11


### ساخت ژن های خام
<p dir=rtl>
در این قسمت داده های جدول را باید به ژن ها تبدیل کنیم اما مقدار وزن و فعال بودن آن در کروموزوم تعیین می شود
</p>

In [244]:
row_gens = list[Gen]()
for i in snacks.iterrows():
    name, max_weight, value = i[1]
    row_gens.append(Gen(name, max_weight, value/max_weight))

In [245]:
for i in row_gens[:2]:
    print(i)

------------
name: MazMaz
status: False
weight: 0
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: False
weight: 0
max weight: 15
vpw: 0.67


### ساخت کروموزوم های اولیه
<p dir=rtl>
در این قسمت ژن ها را برای هر کروموزوم مقدار شانسی می دهیم و کروموزوم های اولیه را می سازیم. برای تعداد جمعیت پارامتر هایی تعریف می کنیم.
</p>

<p dir=rtl>
تابع زیر با استفاده از ژن های خام یک کروموزوم رندوم می سازد
</p>

In [246]:
def create_random_chromosome() -> Chromosome:
    gens = list[Gen]()
    for i in row_gens:
        new_gen = copy.deepcopy(i)
        new_gen.set_random_gen()
        gens.append(new_gen)
    return Chromosome(gens)

<p dir=rtl>
تابع زیر با کمک تابع قبل کروموزوم های اولیه را می سازد
</p>

In [247]:
def create_first_chromosomes() -> list[Chromosome]:
    first_chromosomes = list[Chromosome]()
    for _ in range(FIRST_CHROMOSOMES_COUNT):
        first_chromosomes.append(create_random_chromosome())
    return first_chromosomes

<p dir=rtl>
کارکرد آن را بررسی می کنیم
</p>

In [248]:
test = create_first_chromosomes()
for i in test[:2]:
    print(r'###############')
    print(i)
    print(r'__________')
    for j in i.gens[:2]:
        print(j)

###############
__________
active_count: 11
weight: 108
value: 100.0
vpw: 0.93
__________
------------
name: MazMaz
status: False
weight: 7.8
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: True
weight: 14.2
max weight: 15
vpw: 0.67
###############
__________
active_count: 6
weight: 63
value: 61.0
vpw: 0.97
__________
------------
name: MazMaz
status: False
weight: 7.8
max weight: 10
vpw: 1.00
------------
name: Doogh-e-Abali
status: False
weight: 2.3
max weight: 15
vpw: 0.67
